<a href="https://colab.research.google.com/github/EloizioHMD/Python_DS/blob/main/Scrapping_BeautifulSoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrapping com Python e BeautifulSoup

Quando usar o Scrapping? Quando o site não possuí serviços de acesso informação de uma site, como API, por exemplo.

Beautiful Soup é uma biblioteca que facilita a extração de informações de páginas da web. Ele fica no topo de um analisador HTML ou XML, fornecendo idiomas Pythonic para iterar, pesquisar e modificar a árvore de análise.

# Conhecimentos importantes
## Estrutura DOM (Document Object Model)

O Modelo de Documento por Objetos (do inglês Document Object Model - DOM) é uma convenção multiplataforma e independente de linguagem de programação, fiscalizada pelo entidade World Wide Web Consortium (W3C), para representação e interação com objetos em documentos HTML, XHTML e, XML. Onde os elementos/nós de cada documento são organizados em uma estrutura de árvore, chamada de Árvore DOM, que endereça e manipula via uso de funções/métodos (interface pública) sobre os objetos, especificada de acordo com a interface de programação de aplicações (API) utilizada, que oferece uma maneira padrão de se acessar cada elemento de um documento, criando páginas altamente dinâmicas. [Wikipédia - Modelo de Objeto de Documentos](https://pt.wikipedia.org/wiki/Modelo_de_Objeto_de_Documentos)

<p align="center">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e4/JKDOM.SVG/360px-JKDOM.SVG.png#center" 
        alt="Hierarquia dos objetos num exemplo HTML DOM - Document Object Mode" 
        width="360" 
        height="480"/>
</p>

### Exemplo de extração de dados de páginas web

Nesse exemplo os dados de todos os imóveis a venda no site [VivaReal](https://www.vivareal.com.br/venda/?pagina=5) e escreve os dados extraidos num banco de dados (SQLIte).

Na última verificação, o site tinha 277 páginas de imóveis, 36 imóveis por página. Se deixá-lo rodar integralmente, teremos no final quase 10000 imóveis.

In [4]:
#bibliotecas que iremos usar
#pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [5]:
url = 'https://www.vivareal.com.br/venda/?pagina={npagina}' # url com tag de iteração
userAgents = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36']

## Análise inicial de cada elemento da página

Este trecho serve para testarmos a extração de cada elemento desejado, seja nome do imóvel, preço etc. Quando compreendido, movemos o trecho de código finalizado para o bloco mais adiante no notebook, em que o scrapping é feito em lote.

### Passo 1: Traz o HTML do site

Use o `requests` para fazer uma operação HTTP GET e obter o HTML da página.

In [6]:
url.format( npagina=2 )

'https://www.vivareal.com.br/venda/?pagina=2'

In [8]:
doc = requests.get( url.format(npagina=1), headers={'User-agent':userAgents[0]} )

Vamos verificar o que o webserver retornou olhando só os 700 primeiros caracteres (variavel `olhadinha`).

In [9]:
doc.content[:700]

b' <!DOCTYPE html> <html lang="pt-BR" prefix="og: https://ogp.me/ns#"> <head> <meta name="msvalidate.01" content="5DB2670D7BB1D1D3E36C46F7C3D59380"> <meta name="omniverify" content="omni2c019e0"> <meta property="og:url" content="https://www.vivareal.com.br/venda/"> <meta property="og:type" content=""> <meta property="og:image" content=""> <meta property="og:image:type" content="image/jpeg"> <meta property="og:image:secure_url" content="">  <meta property="og:title" content="Comprar Im\xc3\xb3veis - Oferta de im\xc3\xb3veis \xc3\xa0 venda - Viva Real"> <meta property="og:description" content="Mais de 5.963.473 im\xc3\xb3veis para comprar no Brasil? No Viva Real voc\xc3\xaa acha a maior oferta de im\xc3\xb3veis \xc3\xa0 venda nas melhor'

### Passo 2: Analiza o texto HTML

A biblioteca `BeautifulSoup` tem a capacidade de analizar o HTML entregue pelo site (que não é nada mais que texto corrido, como vimos), e convertê-lo em DOM (document object model). O DOM é o mesmo documento HTML só que todas as tags e sua hierarquia foram identificadas ao ponto de podermos fazer buscas por tags e attributos de seus tags.

* Documentação: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [10]:
analisador = BeautifulSoup(doc.content, 'html.parser')

In [11]:
analisador

 <!DOCTYPE html>
 <html lang="pt-BR" prefix="og: https://ogp.me/ns#"> <head> <meta content="5DB2670D7BB1D1D3E36C46F7C3D59380" name="msvalidate.01"/> <meta content="omni2c019e0" name="omniverify"/> <meta content="https://www.vivareal.com.br/venda/" property="og:url"/> <meta content="" property="og:type"/> <meta content="" property="og:image"/> <meta content="image/jpeg" property="og:image:type"/> <meta content="" property="og:image:secure_url"/> <meta content="Comprar Imóveis - Oferta de imóveis à venda - Viva Real" property="og:title"/> <meta content="Mais de 5.963.473 imóveis para comprar no Brasil? No Viva Real você acha a maior oferta de imóveis à venda nas melhores imobiliárias do Brasil." property="og:description"/> <meta content="Mais de 5.963.473 imóveis para comprar no Brasil? No Viva Real você acha a maior oferta de imóveis à venda nas melhores imobiliárias do Brasil." name="description"/> <title> Comprar Imóveis - Oferta de imóveis à venda - Viva Real </title> <link href="htt

A variável `analisador` agora contém o DOM do texto retornado pelo web server. Então agora usamos ela para buscar tags específicas.

Usamos o _inspector_ do browser para entender a estrutura do documento que o HTMLeiro da VivaReal concebeu. Descobrimos que os imóveis estão publicados assim:

```html
<div id="12345">
    <div class="js-card-selector">
        ...mais tags sobre o imóvel...
    </div>
</div>

<div id="54321">
    <div class="js-card-selector">
        ...mais tags sobre o imóvel...
    </div>
</div>
```

Cada imóvel está contido dentro de seu respectivo `<div id="....">`, mas não temos nada específico e genérico nesta tag para selecioná-la. Já a tag imediatamente interior, `<div class="js-card-selector">`, é idêntica para cada imóvel e contém a classe `js-card-selector`, o que a torna uma ótima candidata para ser selecionada.

Então abaixo criaremos uma lista chamada `imoveis` que contém todos os trechos DOM que respeitam o nosso filtro: `<div>`s com classe `js-card-selector`.

In [12]:
imoveis = analisador.find_all('div', class_='js-card-selector')

In [13]:
len(imoveis)

36

Vamos dar uma olhadinha no que tem dentro de cada ítem dos cinco primeiros imóveis

In [14]:
imoveis[0]

<div class="js-card-selector"> <article class="property-card__container js-property-card" data-see-phone=""> <div class="property-card__main-info"> <div class="property-card__main-link"> <div class="property-card__carousel js-property-carousel "> <a class="property-card__labels-container js-main-info js-listing-labels-link" href="/imovel/apartamento-2-quartos-marape-bairros-santos-com-garagem-64m2-venda-RS540000-id-2580913929/"> <div class="property-card__inactive-listing">Indisponível</div> <div class="property-card__already-seen">Visualizado</div> </a> <div class="carousel__container js-carousel-scroll" style="width: calc(5 * 100%);"> <div class="carousel__item-wrapper js-carousel-item-wrapper"> <img alt="Foto 1 de Apartamento com 2 Quartos à venda, 64m² em Marapé, Santos" class="carousel__image js-carousel-image" src="https://resizedimgs.vivareal.com/crop/286x200/named.images.sp/8c5aa5ac2d4eefa4e246e3980b854ac9/foto-1-de-apartamento-com-2-quartos-a-venda-64m-em-marape-santos.jpg" ti

### Passo 3: Monto receitas para extrair cada dado que desejo

Ótimo. Agora olhando a estrutura de tags e atributos, uso seletores para isolar exatamente o dado que desejo. Após isolá-lo, ainda forço ele a passar por filtros para eliminar inutilidades como espaços (uso `strip()`), prefixos como "R$ " (uso `replace("R$ ","")`) e ajusto o ponto decimal de números (uso `replace(".","")`) para em seguida convertê-los de texto para números (uso `int()`).

É muito importante para mim também obter o ID do imóvel. Como esta informação está num atrbuto do `<div>` pai, uso o seletor `parent`.

In [15]:
imoveis[0].find( 'span', class_='property-card__title js-cardLink js-card-title' ).text.strip()

'Apartamento com 2 Quartos à Venda, 64m²'

In [19]:
imoveis[0].find( 'span', class_='property-card__address' ).text.strip()

'Avenida Doutor Moura Ribeiro, 125 - Marapé, Santos - SP'

In [20]:
imoveis[0].find( 'div', class_='property-card__price js-property-card-prices js-property-card__price-small' ).text.strip().replace('R$', '').replace('.', '')

' 540000'

In [21]:
imoveis[0].find('span', class_='property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area').text.strip()

'64'

## Extração em massa

Depois da análise acima, podemos criar nosso loop que consome todo o site. Página por página, imóvel por imóvel, dado por dado.

In [22]:
paginas_teste = 20

In [23]:
fichas = []

for pagina_teste in range(1, paginas_teste):
  print('Pagina', url.format(npagina=pagina_teste))
  
  doc = requests.get( url.format(npagina=pagina_teste), headers={'User-agent':userAgents[0]} )
  analisador = BeautifulSoup(doc.content, 'html.parser')

  imoveis = analisador.find_all( 'div', class_='js-card-selector')

  for unidade in imoveis:
    uni = {}

    uni['titulo'] = unidade.find( 'span', class_='property-card__title js-cardLink js-card-title' ).text.strip()
    uni['endereço'] = unidade.find( 'span', class_='property-card__address' ).text.strip()
    try:
      uni['preço'] = unidade.find( 'div', class_='property-card__price js-property-card-prices js-property-card__price-small' ).text.strip().replace('R$', '').replace('.', '')
    except:
      uni['preço'] = -1
    uni['metragem'] = unidade.find('span', class_='property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area').text.strip()
    fichas.append(uni)

Pagina https://www.vivareal.com.br/venda/?pagina=1
Pagina https://www.vivareal.com.br/venda/?pagina=2
Pagina https://www.vivareal.com.br/venda/?pagina=3
Pagina https://www.vivareal.com.br/venda/?pagina=4
Pagina https://www.vivareal.com.br/venda/?pagina=5
Pagina https://www.vivareal.com.br/venda/?pagina=6
Pagina https://www.vivareal.com.br/venda/?pagina=7
Pagina https://www.vivareal.com.br/venda/?pagina=8
Pagina https://www.vivareal.com.br/venda/?pagina=9
Pagina https://www.vivareal.com.br/venda/?pagina=10
Pagina https://www.vivareal.com.br/venda/?pagina=11
Pagina https://www.vivareal.com.br/venda/?pagina=12
Pagina https://www.vivareal.com.br/venda/?pagina=13
Pagina https://www.vivareal.com.br/venda/?pagina=14
Pagina https://www.vivareal.com.br/venda/?pagina=15
Pagina https://www.vivareal.com.br/venda/?pagina=16
Pagina https://www.vivareal.com.br/venda/?pagina=17
Pagina https://www.vivareal.com.br/venda/?pagina=18
Pagina https://www.vivareal.com.br/venda/?pagina=19


Processei todos os imóveis de todas as páginas. Vamos ver o resultado...

In [24]:
fichas

[{'titulo': 'Apartamento com 2 Quartos à Venda, 64m²',
  'endereço': 'Avenida Doutor Moura Ribeiro, 125 - Marapé, Santos - SP',
  'preço': ' 540000',
  'metragem': '64'},
 {'titulo': 'Apartamento com 2 Quartos à Venda, 50m²',
  'endereço': 'Rua Homero de Miranda Gomes, 829 - Fundos, Biguaçu - SC',
  'preço': ' 155000',
  'metragem': '50'},
 {'titulo': 'Apartamento com 3 Quartos à Venda, 162m²',
  'endereço': 'Rodovia João Paulo, 2301 - Joao Paulo, Florianópolis - SC',
  'preço': ' 4000000',
  'metragem': '162'},
 {'titulo': 'Apartamento com 2 Quartos à Venda, 50m²',
  'endereço': 'Rua Engenheiro José Bueno Bicalho, 425 - Jardim Vergueiro, São Paulo - SP',
  'preço': ' 235000      Preço abaixo do mercado',
  'metragem': '50'},
 {'titulo': 'Apartamento com 3 Quartos à Venda, 96m²',
  'endereço': 'Rua Ossian Terceiro Teles, 263 - Jardim Prudência, São Paulo - SP',
  'preço': ' 880000',
  'metragem': '96'},
 {'titulo': 'Apartamento com 2 Quartos à Venda, 80m²',
  'endereço': 'Rua Raquel Pr

Agora converterei minha lista de dicts para um DataFrame chamado `todosOsImoveis`. Para tal, crio um DataFrame vazio nomeando as colunas com as chaves de um dict `uni`.

In [25]:
df = pd.DataFrame(columns=uni.keys())
df

,titulo,endereço,preço,metragem


In [26]:
df = df.append(fichas)
df.head()

,titulo,endereço,preço,metragem
0,"Apartamento com 2 Quartos à Venda, 64m²","Avenida Doutor Moura Ribeiro, 125 - Marapé, Sa...",540000,64
1,"Apartamento com 2 Quartos à Venda, 50m²","Rua Homero de Miranda Gomes, 829 - Fundos, Big...",155000,50
2,"Apartamento com 3 Quartos à Venda, 162m²","Rodovia João Paulo, 2301 - Joao Paulo, Florian...",4000000,162
3,"Apartamento com 2 Quartos à Venda, 50m²","Rua Engenheiro José Bueno Bicalho, 425 - Jardi...",235000 Preço abaixo do mercado,50
4,"Apartamento com 3 Quartos à Venda, 96m²","Rua Ossian Terceiro Teles, 263 - Jardim Prudên...",880000,96


In [27]:
df.shape

(684, 4)